In [1]:
import time
import json
import redis
import asyncio
import numpy as np
from aredis import StrictRedis

In [2]:
rd = redis.Redis(host='127.0.0.1', port=6379, db=8)
ar = StrictRedis(host='127.0.0.1', port=6379, db=8)

In [3]:

for key in rd.keys():
    rd.delete(key)

In [4]:
for key in rd.keys():
    print(key)

In [5]:
await asyncio.sleep(int(time.mktime(time.strptime(f'{time.strftime("%Y-%m-%d")} 09:13:30', '%Y-%m-%d %H:%M:%S'))) - time.time()) 

date = time.strftime('%Y%m%d')

check_points_length = int(rd.get(f'hq_{date}_check_points_length'))

assist_count = int(rd.get('hq_assist_count'))

msg = {
    'command': 'snapshotting',
    'date': time.strftime('%Y%m%d')
}

for _ in range(assist_count):
    rd.lpush(f'hq_assist_{_}', json.dumps(msg))

In [6]:
await ar.pubsub_channels()

[]

In [7]:
async def snapshot_handler(assist_count, check_points_length):
    status = np.zeros(check_points_length, dtype=int)
    status

    def message_handler(message):

        assist_idx = str(message['channel']).split('_')[2]
        data = json.loads(message['data'])

        if data['status'] == 'successful':
            check_point_idx = int(data['idx'])
            check_point = data['check_point']
            status[check_point_idx] += 1

            if status[check_point_idx] == assist_count:
                pass

            print('handler:', assist_idx, status[check_point_idx], message['data'])
            
    p = ar.pubsub()
    await p.psubscribe(**{'hq_assist_*_snapshotting': message_handler})

    while status[-1] != assist_count:

        message = await p.get_message()

#         if np.sum(status) >= assist_count:
#             break

    await p.punsubscribe('hq_assist_*_snapshotting')
    p.close()

In [ ]:
await snapshot_handler(assist_count, check_points_length)

handler: 0 1 b'{"status": "successful", "idx": 0, "check_point": 1624410890}'
handler: 3 2 b'{"status": "successful", "idx": 0, "check_point": 1624410890}'
handler: 1 3 b'{"status": "successful", "idx": 0, "check_point": 1624410890}'
handler: 2 4 b'{"status": "successful", "idx": 0, "check_point": 1624410890}'
handler: 6 5 b'{"status": "successful", "idx": 0, "check_point": 1624410890}'
handler: 4 6 b'{"status": "successful", "idx": 0, "check_point": 1624410890}'
handler: 5 7 b'{"status": "successful", "idx": 0, "check_point": 1624410890}'
handler: 0 1 b'{"status": "successful", "idx": 1, "check_point": 1624410900}'
handler: 2 2 b'{"status": "successful", "idx": 1, "check_point": 1624410900}'
handler: 4 3 b'{"status": "successful", "idx": 1, "check_point": 1624410900}'
handler: 3 4 b'{"status": "successful", "idx": 1, "check_point": 1624410900}'
handler: 1 5 b'{"status": "successful", "idx": 1, "check_point": 1624410900}'
handler: 6 6 b'{"status": "successful", "idx": 1, "check_point":

handler: 6 1 b'{"status": "successful", "idx": 15, "check_point": 1624410970}'
handler: 2 2 b'{"status": "successful", "idx": 15, "check_point": 1624410970}'
handler: 0 3 b'{"status": "successful", "idx": 15, "check_point": 1624410970}'
handler: 1 4 b'{"status": "successful", "idx": 15, "check_point": 1624410970}'
handler: 5 5 b'{"status": "successful", "idx": 15, "check_point": 1624410970}'
handler: 3 6 b'{"status": "successful", "idx": 15, "check_point": 1624410970}'
handler: 4 7 b'{"status": "successful", "idx": 15, "check_point": 1624410970}'
handler: 6 1 b'{"status": "successful", "idx": 16, "check_point": 1624410975}'
handler: 3 2 b'{"status": "successful", "idx": 16, "check_point": 1624410975}'
handler: 2 3 b'{"status": "successful", "idx": 16, "check_point": 1624410975}'
handler: 0 4 b'{"status": "successful", "idx": 16, "check_point": 1624410975}'
handler: 1 5 b'{"status": "successful", "idx": 16, "check_point": 1624410975}'
handler: 5 6 b'{"status": "successful", "idx": 16, "

handler: 3 1 b'{"status": "successful", "idx": 30, "check_point": 1624411045}'
handler: 6 2 b'{"status": "successful", "idx": 30, "check_point": 1624411045}'
handler: 0 3 b'{"status": "successful", "idx": 30, "check_point": 1624411045}'
handler: 2 4 b'{"status": "successful", "idx": 30, "check_point": 1624411045}'
handler: 1 5 b'{"status": "successful", "idx": 30, "check_point": 1624411045}'
handler: 4 6 b'{"status": "successful", "idx": 30, "check_point": 1624411045}'
handler: 5 7 b'{"status": "successful", "idx": 30, "check_point": 1624411045}'
handler: 6 1 b'{"status": "successful", "idx": 31, "check_point": 1624411050}'
handler: 3 2 b'{"status": "successful", "idx": 31, "check_point": 1624411050}'
handler: 1 3 b'{"status": "successful", "idx": 31, "check_point": 1624411050}'
handler: 2 4 b'{"status": "successful", "idx": 31, "check_point": 1624411050}'
handler: 4 5 b'{"status": "successful", "idx": 31, "check_point": 1624411050}'
handler: 5 6 b'{"status": "successful", "idx": 31, "

handler: 3 1 b'{"status": "successful", "idx": 45, "check_point": 1624411120}'
handler: 4 2 b'{"status": "successful", "idx": 45, "check_point": 1624411120}'
handler: 2 3 b'{"status": "successful", "idx": 45, "check_point": 1624411120}'
handler: 1 4 b'{"status": "successful", "idx": 45, "check_point": 1624411120}'
handler: 5 5 b'{"status": "successful", "idx": 45, "check_point": 1624411120}'
handler: 6 6 b'{"status": "successful", "idx": 45, "check_point": 1624411120}'
handler: 0 7 b'{"status": "successful", "idx": 45, "check_point": 1624411120}'
handler: 6 1 b'{"status": "successful", "idx": 46, "check_point": 1624411125}'
handler: 5 2 b'{"status": "successful", "idx": 46, "check_point": 1624411125}'
handler: 0 3 b'{"status": "successful", "idx": 46, "check_point": 1624411125}'
handler: 3 4 b'{"status": "successful", "idx": 46, "check_point": 1624411125}'
handler: 4 5 b'{"status": "successful", "idx": 46, "check_point": 1624411125}'
handler: 2 6 b'{"status": "successful", "idx": 46, "

handler: 2 1 b'{"status": "successful", "idx": 60, "check_point": 1624411195}'
handler: 3 2 b'{"status": "successful", "idx": 60, "check_point": 1624411195}'
handler: 5 3 b'{"status": "successful", "idx": 60, "check_point": 1624411195}'
handler: 1 4 b'{"status": "successful", "idx": 60, "check_point": 1624411195}'
handler: 6 5 b'{"status": "successful", "idx": 60, "check_point": 1624411195}'
handler: 4 6 b'{"status": "successful", "idx": 60, "check_point": 1624411195}'
handler: 0 7 b'{"status": "successful", "idx": 60, "check_point": 1624411195}'
handler: 2 1 b'{"status": "successful", "idx": 61, "check_point": 1624411200}'
handler: 0 2 b'{"status": "successful", "idx": 61, "check_point": 1624411200}'
handler: 1 3 b'{"status": "successful", "idx": 61, "check_point": 1624411200}'
handler: 4 4 b'{"status": "successful", "idx": 61, "check_point": 1624411200}'
handler: 3 5 b'{"status": "successful", "idx": 61, "check_point": 1624411200}'
handler: 6 6 b'{"status": "successful", "idx": 61, "

...

handler: 1 1 b'{"status": "successful", "idx": 1500, "check_point": 1624418685}'
handler: 6 2 b'{"status": "successful", "idx": 1500, "check_point": 1624418685}'
handler: 4 3 b'{"status": "successful", "idx": 1500, "check_point": 1624418685}'
handler: 5 4 b'{"status": "successful", "idx": 1500, "check_point": 1624418685}'
handler: 0 5 b'{"status": "successful", "idx": 1500, "check_point": 1624418685}'
handler: 2 6 b'{"status": "successful", "idx": 1500, "check_point": 1624418685}'
handler: 3 7 b'{"status": "successful", "idx": 1500, "check_point": 1624418685}'
handler: 1 1 b'{"status": "successful", "idx": 1501, "check_point": 1624418690}'
handler: 0 2 b'{"status": "successful", "idx": 1501, "check_point": 1624418690}'
handler: 6 3 b'{"status": "successful", "idx": 1501, "check_point": 1624418690}'
handler: 4 4 b'{"status": "successful", "idx": 1501, "check_point": 1624418690}'
handler: 3 5 b'{"status": "successful", "idx": 1501, "check_point": 1624418690}'
handler: 5 6 b'{"status": "s

handler: 1 1 b'{"status": "successful", "idx": 1515, "check_point": 1624418760}'
handler: 2 2 b'{"status": "successful", "idx": 1515, "check_point": 1624418760}'
handler: 6 3 b'{"status": "successful", "idx": 1515, "check_point": 1624418760}'
handler: 4 4 b'{"status": "successful", "idx": 1515, "check_point": 1624418760}'
handler: 0 5 b'{"status": "successful", "idx": 1515, "check_point": 1624418760}'
handler: 3 6 b'{"status": "successful", "idx": 1515, "check_point": 1624418760}'
handler: 5 7 b'{"status": "successful", "idx": 1515, "check_point": 1624418760}'
handler: 2 1 b'{"status": "successful", "idx": 1516, "check_point": 1624418765}'
handler: 4 2 b'{"status": "successful", "idx": 1516, "check_point": 1624418765}'
handler: 6 3 b'{"status": "successful", "idx": 1516, "check_point": 1624418765}'
handler: 0 4 b'{"status": "successful", "idx": 1516, "check_point": 1624418765}'
handler: 3 5 b'{"status": "successful", "idx": 1516, "check_point": 1624418765}'
handler: 5 6 b'{"status": "s

handler: 4 5 b'{"status": "successful", "idx": 1529, "check_point": 1624418830}'
handler: 5 6 b'{"status": "successful", "idx": 1529, "check_point": 1624418830}'
handler: 1 7 b'{"status": "successful", "idx": 1529, "check_point": 1624418830}'
handler: 5 1 b'{"status": "successful", "idx": 1530, "check_point": 1624418835}'
handler: 0 2 b'{"status": "successful", "idx": 1530, "check_point": 1624418835}'
handler: 6 3 b'{"status": "successful", "idx": 1530, "check_point": 1624418835}'
handler: 3 4 b'{"status": "successful", "idx": 1530, "check_point": 1624418835}'
handler: 1 5 b'{"status": "successful", "idx": 1530, "check_point": 1624418835}'
handler: 4 6 b'{"status": "successful", "idx": 1530, "check_point": 1624418835}'
handler: 2 7 b'{"status": "successful", "idx": 1530, "check_point": 1624418835}'
handler: 1 1 b'{"status": "successful", "idx": 1531, "check_point": 1624418840}'
handler: 6 2 b'{"status": "successful", "idx": 1531, "check_point": 1624418840}'
handler: 4 3 b'{"status": "s

handler: 5 7 b'{"status": "successful", "idx": 1544, "check_point": 1624418905}'
handler: 0 1 b'{"status": "successful", "idx": 1545, "check_point": 1624418910}'
handler: 3 2 b'{"status": "successful", "idx": 1545, "check_point": 1624418910}'
handler: 1 3 b'{"status": "successful", "idx": 1545, "check_point": 1624418910}'
handler: 2 4 b'{"status": "successful", "idx": 1545, "check_point": 1624418910}'
handler: 6 5 b'{"status": "successful", "idx": 1545, "check_point": 1624418910}'
handler: 4 6 b'{"status": "successful", "idx": 1545, "check_point": 1624418910}'
handler: 5 7 b'{"status": "successful", "idx": 1545, "check_point": 1624418910}'
handler: 0 1 b'{"status": "successful", "idx": 1546, "check_point": 1624418915}'
handler: 2 2 b'{"status": "successful", "idx": 1546, "check_point": 1624418915}'
handler: 1 3 b'{"status": "successful", "idx": 1546, "check_point": 1624418915}'
handler: 6 4 b'{"status": "successful", "idx": 1546, "check_point": 1624418915}'
handler: 5 5 b'{"status": "s

handler: 0 1 b'{"status": "successful", "idx": 1560, "check_point": 1624418985}'
handler: 3 2 b'{"status": "successful", "idx": 1560, "check_point": 1624418985}'
handler: 4 3 b'{"status": "successful", "idx": 1560, "check_point": 1624418985}'
handler: 5 4 b'{"status": "successful", "idx": 1560, "check_point": 1624418985}'
handler: 1 5 b'{"status": "successful", "idx": 1560, "check_point": 1624418985}'
handler: 2 6 b'{"status": "successful", "idx": 1560, "check_point": 1624418985}'
handler: 6 7 b'{"status": "successful", "idx": 1560, "check_point": 1624418985}'
handler: 1 1 b'{"status": "successful", "idx": 1561, "check_point": 1624418990}'
handler: 0 2 b'{"status": "successful", "idx": 1561, "check_point": 1624418990}'
handler: 3 3 b'{"status": "successful", "idx": 1561, "check_point": 1624418990}'
handler: 6 4 b'{"status": "successful", "idx": 1561, "check_point": 1624418990}'
handler: 4 5 b'{"status": "successful", "idx": 1561, "check_point": 1624418990}'
handler: 5 6 b'{"status": "s

In [ ]:
date = time.strftime('%Y%m%d')
date

In [ ]:
check_points_length = int(rd.get(f'hq_{date}_check_points_length'))
check_points_length

In [ ]:
assist_count = int(rd.get('hq_assist_count'))
assist_count

In [ ]:
msg = {
    'command': 'quit'
}


In [ ]:
msg = {
    'command': 'snapshotting',
    'date': time.strftime('%Y%m%d')
}
msg

In [ ]:
for _ in range(assist_count):
    rd.lpush(f'hq_assist_{_}', json.dumps(msg))

In [ ]:
async def incremental_save(assist_count, check_points_length):
    subs = [ ar.pubsub() for assist_idx in range(assist_count) ]
    
    results = await asyncio.gather(*[ sub.subscribe(f'hq_assist_{_}_snapshotting') for _,sub in enumerate(subs) ])
    results = await asyncio.gather(*[ sub.parse_response() for sub in subs ])
    assert all([ result[0] == b'subscribe' and result[-1]==1 for result in results])
    
    date = time.strftime('%Y%m%d')
    while True:
        results = await asyncio.gather(*[ sub.parse_response() for sub in subs ])
        idx = json.loads(results[0][2])['idx']
        msg = {
            'command': 'incremental_save',
            'date': time.strftime('%Y%m%d'),
            'idx': idx
        }
        rd.lpush(f'hq_assist_0', json.dumps(msg))
        
        if idx+1 >= check_points_length:
            break
    
    results = await asyncio.gather(*[ sub.unsubscribe(f'hq_assist_{_}_snapshotting') for _,sub in enumerate(subs) ])
    results = await asyncio.gather(*[ sub.parse_response() for sub in subs ])
    assert all([ result[0] == b'unsubscribe' and result[-1]==0 for result in results])

In [ ]:
asyncio.create_task(incremental_save(assist_count, check_points_length))